In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Set up the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Add this line if you want to run Chrome in headless mode
chrome_path = "path/to/chromedriver"  # Update with your chromedriver path

# Initialize ChromeService with the executable path
chrome_service = ChromeService(chrome_path)

# Pass ChromeService to webdriver
driver = webdriver.Chrome(service=chrome_service, options=options)

url = "https://www.nepalstock.com.np/floor-sheet"
driver.get(url)

NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
# Wait for the table to be present
table = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "table__lg"))
)


In [ ]:
# Function to extract table data
def extract_table_data(table):
    data = []
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        row_data = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        if row_data:
            data.append(row_data)
    return data


In [ ]:
# Extract table data from the current page
table_data = extract_table_data(table)


In [ ]:
# Function to go to the next page
def go_to_next_page():
    try:
        next_button = driver.find_element(By.CLASS_NAME, "pagination-next")
        next_button.click()
        return True
    except:
        return False


In [ ]:
# Extract data from all pages
all_data = []
while True:
    all_data.extend(table_data)
    if not go_to_next_page():
        break
    time.sleep(2)  # Add a delay to ensure the page loads before extracting data
    table_data = extract_table_data(table)


In [ ]:
# Close the WebDriver
driver.quit()


In [ ]:
# Convert the data to a Pandas DataFrame
columns = ["SN", "Contract No.", "Stock Symbol", "Buyer", "Seller", "Quantity", "Rate (Rs)", "Amount (Rs)"]
df = pd.DataFrame(all_data, columns=columns)


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("output_data.csv", index=False)


In [ ]:
print("Data has been scraped and saved to 'output_data.csv'")